In [2]:
from langchain.document_loaders import WebBaseLoader

In [3]:
njs_base_docs_url = "https://nextjs.org/"

In [4]:
import requests
from bs4 import BeautifulSoup

URL = "https://nextjs.org/docs"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

In [5]:
# find all links

links = soup.find_all("a")

In [6]:
# filter to get only links with href to docs

links = [link for link in links if link["href"].startswith("/docs")]

In [7]:
loader = WebBaseLoader(njs_base_docs_url)
loader.load()

[Document(page_content='Next.js by Vercel - The React Framework for the WebDeploy Next.js on Vercel →Skip to contentNext.jsCopy Logo as SVG Copy Wordmark as SVGShowcaseDocsBlogAnalyticsTemplatesEnterpriseFeedbackEmailFeedbackSendLearnThe React Framework for the WebUsed by some of the world\'s largest companies, Next.js enables you to create full-stack Web applications by extending the latest React features, and integrating powerful Rust-based JavaScript tooling for the fastest builds.Get StartedLearn Next.js▲ ~ npx create-next-app@latestWhat\'s in Next.js?All the tools you need to make the Web. Faster.Built-in OptimizationsAutomatic Image, Font, and Script Optimizations for improved UX and Core Web Vitals.Data FetchingData Fetching methods that run on the server and enable you to render content in different ways.Node.js and Edge RuntimesBuild scalable solutions with Serverless functions, and deliver fast dynamic, personalized content with the Edge.Powerful Routing and LayoutsBuild comp

In [9]:
!pip install nest_asyncio

import nest_asyncio

nest_asyncio.apply()

In [10]:
loaders = [ "https://nextjs.org" + link["href"] for link in links ]
loader = WebBaseLoader(loaders)
loader.request_per_second = 1
docs = loader.aload()

In [14]:
# dir(docs[0])
docs[0].json()


'{"page_content": "Getting Started | Next.jsDeploy Next.js on Vercel \\u2192Skip to contentNext.jsCopy Logo as SVG Copy Wordmark as SVGShowcaseDocsBlogAnalyticsTemplatesEnterpriseFeedbackEmailFeedbackSendLearnMenuDocumentationGetting StartedBasic FeaturesPagesData FetchingOverviewgetServerSidePropsgetStaticPropsgetStaticPathsIncremental Static RegenerationClient sideBuilt-in CSS SupportLayoutsImage OptimizationFont OptimizationStatic File ServingFast RefreshESLintTypeScriptEnvironment VariablesSupported Browsers and FeaturesHandling ScriptsRoutingIntroductionDynamic RoutesImperativelyShallow RoutingAPI RoutesIntroductionDynamic API RoutesRequest HelpersResponse HelpersEdge API RoutesGoing to ProductionDeploymentAuthenticationTestingAccessibilityGuidesBuilding FormsAdvanced FeaturesNext.js CompilerTurbopackPreview ModeDynamic ImportAutomatic Static OptimizationStatic HTML ExportAbsolute Imports and Module Path AliasesUsing MDXAMP SupportIntroductionAdding AMP ComponentsAMP ValidationAMP

In [60]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)

In [63]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [65]:
from langchain.vectorstores import Chroma
db = Chroma.from_documents(texts, embeddings)

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.
Exiting: Cleaning up .chroma directory


In [76]:
db.max_marginal_relevance_search("create a  and CSS", k=3)

[Document(page_content='Gatsby: createPages API inside gatsby-node.js for each blog post, then have a template file at src/templates/blog-post.js.\nNext: Create pages/blog/[slug].js which contains the blog post template. The value of slug is accessible through a query parameter. For example, the route /blog/first-post would forward the query object { \'slug\': \'first-post\' } to pages/blog/[slug].js (learn more here).\n\nStyling\n\n\n\n\n\nWith Gatsby, global CSS imports are included in gatsby-browser.js. With Next, you should create a custom _app.js for global CSS. When migrating, you can copy over your CSS imports directly and update the relative file path, if necessary. Next.js has built-in CSS support.\nLinks\n\n\n\n\n\nThe Gatsby Link and Next.js Link component have a slightly different API.\n// Gatsby\n\nimport { Link } from \'gatsby\'\n\nexport default function Home() {\n  return <Link to="/blog">blog</Link>\n}\n\n// Next.js\n\nimport Link from \'next/link\'\n\nexport default f

In [47]:
from langchain.indexes import VectorstoreIndexCreator
index = VectorstoreIndexCreator().from_loaders(loaders)

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


In [49]:
index.vectorstore

In [40]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

llm = OpenAI(temperature=0.5, model_name="gpt-4")
db = index.vectorstore.as_retriever(search_kwargs={"k": 3})

docs = db.get_relevant_documents("""
Create a new page component.
""")
docs2 = db.get_relevant_documents("""
Create CSS for a page component.
""")
docs3 = db.get_relevant_documents("""
Create a dynamic API route.
""")

# llm = OpenAI(temperature="0.5", model_name="gpt-4")
# chain = load_qa_chain(llm, chain_type="stuff")
# chain.run(input_documents=docs, question=query)

/Users/forrestmurray/mambaforge/lib/python3.10/site-packages/langchain/llms/openai.py:169: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/forrestmurray/mambaforge/lib/python3.10/site-packages/langchain/llms/openai.py:608: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
